In [1]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [1]:
import os
print(os.getcwd())       # 현재 디렉터리 경로
print(os.listdir())      # 현재 디렉터리 파일 목록


/home/e2c9053a-079d-4d40-8e7f-5fb9d860d7c4
['submission_SegmentCDE.csv', 'submission_SegmentAB.csv', '.ipynb_checkpoints', '.virtualenvs', 'Catboost_fin3.ipynb', '.vimrc', 'Catboost_fin2.ipynb', '.bashrc', 'Untitled.ipynb', '.jupyter', 'base_clean_train.csv', '.local', 'anaconda_projects', '.profile', '.pythonstartup.py', '수정1트.ipynb', 'tmp_base_clean_test.parquet', 'README.ipynb', '.gitconfig', 'saved_models', 'Catboost_final_ed.ipynb', '.npm', 'base_clean_test.csv', 'Catboost_fin.ipynb', '.ipython', '.config', '.anaconda', 'hyperclova_api.py', '.cache', 'finance_recommender_app_updated.py']


In [3]:
%%writefile hyperclova_api.py
import os
from functools import lru_cache

API_KEY = os.getenv("HCX_API_KEY")
BASE_URL = "https://api.hyperclova.ai/v1"

def _get_headers():
    if not API_KEY:
        raise ValueError("HCX_API_KEY 환경변수를 설정해주세요.")
    return {"Authorization":f"Bearer {API_KEY}", "Content-Type":"application/json"}

@lru_cache(maxsize=128)
def classify_content(text: str) -> dict:
    return {"risk":"중립","themes":[],"period":"장기"}

@lru_cache(maxsize=128)
def summarize_text(text: str, max_length: int = 200) -> str:
    return text[:max_length] + "..."

def chat_completion(prompt: str) -> str:
    return f"[HyperCLOVA 응답 예시] {prompt}"


Overwriting hyperclova_api.py


In [4]:
%%writefile finance_recommender_app_updated.py
# -*- coding: utf-8 -*-
import requests, json, os, hashlib
from datetime import datetime
from collections import Counter
import streamlit as st
from hyperclova_api import classify_content, summarize_text, chat_completion

# ----------------------------
# 설정 및 상수
# ----------------------------
CLIENT_ID = "r139rV3OFmXfYuZj2geF"
CLIENT_SECRET = "tcwu4YHfNT"
NEWS_DB_PATH = "news_data.json"
PROFILE_DB_PATH = "profile_db.json"
HYPERCLOVA_CACHE = {}

# ----------------------------
# 유틸 함수
# ----------------------------
def today_str():
    return datetime.now().strftime("%Y-%m-%d")

def make_hash(title, link):
    return hashlib.md5((title + link).encode("utf-8")).hexdigest()

def load_json(filepath):
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_json(obj, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

# ----------------------------
# 뉴스 크롤링 & 저장
# ----------------------------
def get_news(query, display=20, start=1):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {"query": query, "display": display, "start": start, "sort": "date"}
    try:
        r = requests.get(url, headers=headers, params=params, timeout=5)
        r.raise_for_status()
        return r.json().get("items", [])
    except Exception as e:
        st.error(f"뉴스 API 오류: {e}")
        return []

def crawl_today_news(queries):
    news_db = load_json(NEWS_DB_PATH)
    today = today_str()
    news_db.setdefault(today, [])
    existing = {make_hash(n['title'], n['link']) for n in news_db[today]}
    for q in queries:
        for item in get_news(q):
            h = make_hash(item['title'], item['link'])
            if h not in existing:
                news_db[today].append({
                    "title": item["title"],
                    "description": item["description"],
                    "pubDate": item["pubDate"],
                    "link": item["link"],
                    "query": q
                })
                existing.add(h)
    save_json(news_db, NEWS_DB_PATH)

# ----------------------------
# 투자자 프로필
# ----------------------------
def generate_investor_profile(resp):
    risk_map = {"매우 보수적":0.1,"보수적":0.3,"중립":0.5,"공격적":0.7,"매우 공격적":0.9}
    horizon_map = {"단기":1,"중기":3,"장기":5}
    return {
        "risk_score": risk_map[resp["risk_level"]],
        "horizon_years": horizon_map[resp["horizon"]],
        "interest_tags": resp["interests"]
    }

def update_risk_profile(profile, period=1):
    alpha = min(0.5, 0.1 * period)
    behavior_score = profile["risk_score"]
    new_score = (1-alpha)*profile["risk_score"] + alpha*behavior_score
    profile["risk_score"] = round(new_score, 2)
    return profile

# ----------------------------
# 상품 DB (확장)
# ----------------------------
PRODUCTS_DB = [
    {"name":"EMP 분산 투자 펀드", "risk":0.5,
     "themes":["자산배분","ETF","초분산"], "description":"여러 ETF에 분산투자하여 리스크 관리"},
    {"name":"TDF 2045 시리즈", "risk":0.3,
     "themes":["장기","채권","리밸런싱"], "description":"은퇴 시점별 자동 리밸런싱 펀드"},
    {"name":"글로벌 리츠 펀드", "risk":0.4,
     "themes":["리츠","부동산","배당"], "description":"글로벌 상업용 부동산 투자"},
    {"name":"고정쿠폰 ELS", "risk":0.8,
     "themes":["구조화상품","파생","쿠폰"], "description":"조기상환형 고정쿠폰 ELS"},
    {"name":"Pre-IPO 스타트업 펀드", "risk":0.9,
     "themes":["프리IPO","스타트업","고수익"], "description":"미상장 우량기업 초기 투자"},
    {"name":"Global Infrastructure ETF", "risk":0.6,
     "themes":["인프라","데이터센터","도로"], "description":"세계 인프라 자산 분산"},
    {"name":"Global Timber ETF", "risk":0.6,
     "themes":["임업","ESG","탄소저감"], "description":"임업·펄프 기업 투자"},
    {"name":"AI 스마트베타 ETF", "risk":0.6,
     "themes":["AI","스마트베타","저변동"], "description":"AI 알고리즘 기반 포트폴리오"}
]

def recommend_products(profile, market_themes):
    scored = []
    for prod in PRODUCTS_DB:
        gap = abs(profile["risk_score"] - prod["risk"])
        rs = 1 - gap
        ts = len(set(profile["interest_tags"] + market_themes) & set(prod["themes"])) / (len(prod["themes"])+1e-5)
        score = 0.6*rs + 0.4*ts
        scored.append((score, prod))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [p for _,p in scored[:3]]

# ----------------------------
# Streamlit UI
# ----------------------------
st.set_page_config(page_title="HyperCLOVA 금융추천", layout="wide")
st.title("HyperCLOVA X 기반 맞춤형 금융 상품 추천")

with st.sidebar:
    st.header("투자 성향 설문")
    age = st.slider("연령대",20,60,30)
    risk_level = st.selectbox("위험 성향",["매우 보수적","보수적","중립","공격적","매우 공격적"])
    horizon = st.selectbox("투자 기간",["단기","중기","장기"])
    interests = st.multiselect("관심 분야",["인프라","ETF","TDF","EMP","스마트베타","부동산","프리IPO","구조화"])
    user_keywords = st.multiselect("뉴스 키워드 선택",["AI","금리","인플레이션","인프라","리츠","Pre-IPO","구조화상품","TDF","EMP"])
    if st.button("프로필 생성 & 추천"):
        base_profile = generate_investor_profile({
            "risk_level":risk_level,"horizon":horizon,"interests":interests
        })
        profile = update_risk_profile(base_profile, period=1)
        st.session_state["profile"] = profile
        st.session_state["user_keywords"] = user_keywords
        st.success("프로필 생성 완료!")

if "profile" in st.session_state:
    profile = st.session_state["profile"]
    keys = st.session_state.get("user_keywords") or ["금리","ETF","구조화상품"]
    crawl_today_news(keys)
    news_db = load_json(NEWS_DB_PATH)
    today = max(news_db.keys())
    news_items = news_db[today]

    st.subheader("뉴스 요약 및 태그")
    all_tags = []
    for itm in news_items[:10]:
        text = itm["title"] + " " + itm["description"]
        summary = summarize_text(text)
        tags = classify_content(summary).get("themes",[])
        itm["summary"], itm["tags"] = summary, tags
        all_tags.extend(tags)
        with st.expander(itm["title"]):
            st.write(summary)
            st.caption(f"태그: {', '.join(tags)}")

    market_themes = list(set(st.session_state["user_keywords"] + all_tags))
    recs = recommend_products(profile, market_themes)

    st.subheader("오늘의 추천 금융 상품")
    for p in recs:
        st.markdown(f"### {p['name']} （위험도 {p['risk']}）")
        st.markdown(f"- 설명: {p['description']}")
        st.markdown(f"- 테마: {', '.join(p['themes'])}")

    st.subheader("챗봇 질의응답")
    user_q = st.text_input("궁금한 내용을 입력하세요")
    if user_q:
        prompt = (
            f"사용자 프로필: {profile}\n"
            f"추천 상품: {[r['name'] for r in recs]}\n"
            f"뉴스 키워드: {st.session_state['user_keywords']}\n"
            f"질문: {user_q}\n"
            "친근한 어투로 답변해주세요."
        )
        try:
            answer = chat_completion(prompt)
            st.info(answer)
        except Exception as e:
            st.error(f"Chat API 오류: {e}")


Overwriting finance_recommender_app_updated.py


In [ ]:
!python -m streamlit run finance_recommender_app_updated.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.0.192:8501
  External URL: http://10.0.0.192:8501



In [ ]:
!pip install jupyter-server-proxy

In [ ]:
!streamlit --version

In [ ]:
!streamlit run finance_recommender_app_updated.py